To access gated models on the Hugging Face Hub, you need to authenticate your environment. You can do this by using a Hugging Face API token.

1.  Go to your Hugging Face settings page: [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
2.  Generate a new token or copy an existing one with "read" access.
3.  Run the following cell and paste your token when prompted.

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:

import torch
from types import MethodType
import os
import torch.nn.functional as F

from transformers import AutoTokenizer
import pickle
from tqdm import tqdm
import os
import random
from transformers import AutoTokenizer
# from utils.instructions import INSTRUCTIONS
from jinja2 import Template
import pickle
import numpy as np
from jinja2 import Template
import argparse
import  torch.nn as nn
import json

In [5]:
import json
data_file = "/content/samanantar_hindi.json"
output_matrix_file = "translation_neurons.json"


data = None
try:
    with open(data_file, "r", encoding="utf-8") as f:
        data = json.load(f)
except FileNotFoundError:
    print(f"ERROR: Data file '{data_file}' not found. Please upload it to Colab.")

def build_translation_prompt(instances):
    system_prompt = (
        "<<SYS>>In this task, you are given an English sentence. "
        "Your job is to translate the given sentence into Hindi accurately, "
        "preserving meaning and grammar.<</SYS>>"
    )
    text = "<s> [INST] " + system_prompt
    for i, inst in enumerate(instances):
        # Extract src and tgt, stripping whitespace
        src = inst["input"]["src"].strip()
        tgt = inst["input"]["tgt"].strip()

        # Append the input and expected output (translation)
        text += f"{src} [/INST] {tgt} "

        # If not the last item in the chunk, prepare for the next turn
        if i < len(instances) - 1:
            text += "[INST] "

    return text.strip()


# --- 3. Processing and Output Generation ---
def generate_chunked_output(data, chunk_size=5):
    """Generates the full output by applying build_translation_prompt in chunks."""
    full_output = []
    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        # Use the custom function to build the prompt for the current chunk
        prompt = build_translation_prompt(chunk)
        full_output.append(prompt)

    # Join chunks with extra blank lines for readability in the final output
    return full_output

# Generate and print the output for the given data (which contains 15 pairs)
final_output = generate_chunked_output(data, chunk_size=5)

print(final_output[0])

<s> [INST] <<SYS>>In this task, you are given an English sentence. Your job is to translate the given sentence into Hindi accurately, preserving meaning and grammar.<</SYS>>However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles [/INST] आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था। [INST] Whosoever desires the reward of the world, with Allah is the reward of the world and of the Everlasting Life. Allah is the Hearer, the Seer. [/INST] और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है [INST] The value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness. [/INST] जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले

In [6]:

sub_squence_list = [[518, 25580, 29962],[518, 29914, 25580, 29962],[3532, 14816, 29903, 6778],[529, 829, 14816, 29903, 6778]]



In [7]:
train_message=final_output[:500]

In [12]:
# Set the model name
COLAB_MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using model: {COLAB_MODEL_NAME}")
print(f"Using device: {device}")

Using model: meta-llama/Llama-2-7b-chat-hf
Using device: cuda


In [13]:
# Assuming COLAB_MODEL_NAME is defined in a previous cell

tokenizer = AutoTokenizer.from_pretrained(COLAB_MODEL_NAME)

# Set pad_token if it's None
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer loaded from: {COLAB_MODEL_NAME}")
print(f"Pad token set to: {tokenizer.pad_token}")

Tokenizer loaded from: meta-llama/Llama-2-7b-chat-hf
Pad token set to: </s>


In [14]:
def find_sublist_indices(main_list, sub_list):
    result = []
    sub_len = len(sub_list)

    for i in range(len(main_list) - sub_len + 1):
        if main_list[i:i+sub_len] == sub_list:
            # Add list of indices for this match
            result.append(list(range(i, i + sub_len)))

    return result



In [15]:
if True:
    train_token = []
    indexs = []
    progress_bar = tqdm(total=len(train_message), desc='Train Processing data')
    for i in range(len(train_message)):
        progress_bar.update(1)
        message = train_message[i]

        result = message
        if True:
            input_ids = tokenizer.encode(result)
            index_start = find_sublist_indices(input_ids,sub_squence_list[0])
            index_1 = find_sublist_indices(input_ids,sub_squence_list[1])
            track_index = index_start+index_1
            print(len(index_start),len(index_1))
            lat_list = [item for sublist in track_index for item in sublist]
            indexs.append(sorted(lat_list))
        train_token.append(tokenizer.encode(result))


Train Processing data: 100%|██████████| 500/500 [01:00<00:00,  8.27it/s] 

Train Processing data:  23%|██▎       | 114/500 [00:00<00:00, 567.22it/s]

5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5



Train Processing data:  47%|████▋     | 233/500 [00:00<00:00, 580.96it/s]

5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5



Train Processing data:  70%|███████   | 352/500 [00:00<00:00, 581.71it/s]

5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5



Train Processing data:  82%|████████▏ | 411/500 [00:00<00:00, 568.68it/s]

5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5



Train Processing data:  94%|█████████▎| 468/500 [00:00<00:00, 562.51it/s]

5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5


In [17]:
train_token[0]

[1,
 1,
 29871,
 518,
 25580,
 29962,
 3532,
 14816,
 29903,
 6778,
 797,
 445,
 3414,
 29892,
 366,
 526,
 2183,
 385,
 4223,
 10541,
 29889,
 3575,
 4982,
 338,
 304,
 14240,
 278,
 2183,
 10541,
 964,
 379,
 14108,
 7913,
 2486,
 29892,
 2225,
 29530,
 6593,
 322,
 25437,
 19423,
 829,
 14816,
 29903,
 6778,
 17245,
 29892,
 2621,
 267,
 29892,
 1058,
 471,
 18096,
 292,
 8314,
 29915,
 29879,
 3739,
 7169,
 2330,
 29892,
 1033,
 871,
 748,
 408,
 2215,
 408,
 278,
 12616,
 8394,
 29879,
 988,
 896,
 5714,
 304,
 20303,
 786,
 493,
 29875,
 322,
 19320,
 793,
 518,
 29914,
 25580,
 29962,
 29871,
 31732,
 30489,
 30296,
 31329,
 30296,
 30316,
 30569,
 30603,
 30436,
 30640,
 30269,
 29871,
 30444,
 30569,
 29871,
 30621,
 30269,
 30603,
 29871,
 30714,
 30569,
 30424,
 30603,
 30580,
 29871,
 30444,
 30569,
 29871,
 30489,
 30269,
 31927,
 29871,
 30871,
 30799,
 227,
 168,
 159,
 30580,
 29871,
 31012,
 30424,
 30269,
 30424,
 30569,
 29871,
 30610,
 30269,
 30603,
 30569,
 29871,

In [18]:
indexs[0]

[3,
 4,
 5,
 82,
 83,
 84,
 85,
 247,
 248,
 249,
 290,
 291,
 292,
 293,
 456,
 457,
 458,
 487,
 488,
 489,
 490,
 619,
 620,
 621,
 637,
 638,
 639,
 640,
 691,
 692,
 693,
 738,
 739,
 740,
 741]

In [22]:
model="meta-llama/Llama-2-7b-chat-hf"

In [23]:
model.to(device)
model.train()
criterion = nn.CrossEntropyLoss(reduction="none")
out_data = [[0]*11008]*32
ss=0
progress_bar = tqdm(total=len(train_token), desc='Getting data')
for input_ids,index in zip(train_token, indexs):
    progress_bar.update(1)
    # print(len(input_ids))
    if len(input_ids)>1300:
        ss+=1
        # print(ss)
        continue
    input_index = [i-1 for i in index]
    label_token = [input_ids[i] for i in index]

    input_ids = torch.tensor(input_ids,dtype=torch.int64).unsqueeze(0).to(device)
    label_token = torch.tensor(label_token,dtype=torch.int64).to(device)

    output = model(input_ids)
    loss1 = criterion(output.logits[0, input_index[:28], :], label_token[:28])
    loss2 = criterion(output.logits[0, input_index[28:], :], label_token[28:])

    # 计算平均损失
    # loss = loss1.mean()*0.0001 + loss2.mean()
    loss = loss1.mean() + loss2.mean()
    model.zero_grad()

    loss.backward()
    # print(loss.item())
    for name, param in model.named_parameters():
        if param.grad is not None and "up_proj" in name:
            # print(f'name: {name}, grad:\n{param.grad.shape}')
            layer = int(name.split(".")[2])
            grad = torch.sum(param.grad,dim=1).cpu().tolist()
            out_data[layer] =  [abs(a) + b for a, b in zip(grad, out_data[layer])]


with open(f"/home/songran/project/task-specific-neuron/matrix/{task}/{args.mod}.json","w") as f:
    json.dump(out_data,f)

AttributeError: 'str' object has no attribute 'to'

In [ ]:
from transformers import LlamaForCausalLM, AutoConfig
import torch.nn as nn

# Load the model and config for dynamic size detection
try:
    config = AutoConfig.from_pretrained(COLAB_MODEL_NAME)
    MOCK_FFN_SIZE = config.intermediate_size
    MOCK_LAYERS = config.num_hidden_layers
    print(f"\nModel Config Loaded: Layers - {MOCK_LAYERS}, FFN Size - {MOCK_FFN_SIZE}")
except Exception as e:
    print(f"Failed to load model config from {COLAB_MODEL_NAME}. Check accessibility.")
    raise

print(f"\nLoading model: {COLAB_MODEL_NAME} on {device}")

try:
    model = LlamaForCausalLM.from_pretrained(
        COLAB_MODEL_NAME,
        torch_dtype=torch.bfloat16 # Using bfloat16 for potentially lower memory usage
    )
    # Initialize the output matrix based on the dynamically loaded configuration
    # Ensure out_data is a list of lists, not a list of references to the same list
    out_data = [[0.0] * MOCK_FFN_SIZE for _ in range(MOCK_LAYERS)]
    print(f"Initialized out_data matrix with shape ({MOCK_LAYERS}, {MOCK_FFN_SIZE})")


except Exception as e:
    print(f"Failed to load or initialize model: {e}")
    print("\nCRITICAL: Ensure the model path is correct and accessible.")
    raise

# Move model to device and set to train mode
model.to(device)
model.train() # Set to train mode for gradient computation

criterion = nn.CrossEntropyLoss(reduction="none")

print("Model loaded, moved to device, and set to training mode.")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]


Model Config Loaded: Layers - 32, FFN Size - 11008

Loading model: meta-llama/Llama-2-7b-chat-hf on cuda


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]